In [1]:
import os

In [2]:
%pwd

'd:\\Profiles\\sg2755\\Desktop\\project\\chicken_disease_classification_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Profiles\\sg2755\\Desktop\\project\\chicken_disease_classification_project'

In [65]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    training_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [66]:
from src.CNN_Classifier.constant import *
from src.CNN_Classifier.utils.common import read_yaml,create_directory
import tensorflow as tf

In [67]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directory([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directory([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    




    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directory([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            training_model_path=Path(training.training_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [68]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import numpy as np

In [69]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        # Call properties without parentheses
        return [
            self._create_tb_callbacks,   # No parentheses
            self._create_ckpt_callbacks  # No parentheses
        ]


In [74]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
    
    def train_valid_generator(self):

        # datagenerator_kwargs = dict(
        #     rescale = 1./255,
        #     validation_split=0.20
        # )

        # dataflow_kwargs = dict(
        #     target_size=self.config.params_image_size[:-1],
        #     batch_size=self.config.params_batch_size,
        #     interpolation="bilinear"
        # )

        def generator_to_dataset(generator):
            output_signature = (
                tf.TensorSpec(shape=(None, *generator.image_shape), dtype=tf.float32),
                tf.TensorSpec(shape=(None, generator.num_classes), dtype=tf.float32)
            )
            dataset = tf.data.Dataset.from_generator(
                lambda: generator,
                output_signature=output_signature
            )
            return dataset


        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale = 1./255,
            validation_split=0.20
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        self.valid_dataset = generator_to_dataset(self.valid_generator).repeat()

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                rescale = 1./255,
                validation_split=0.20
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        self.train_dataset = generator_to_dataset(self.train_generator).repeat()

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        # self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        # self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.steps_per_epoch = int(np.ceil(self.train_generator.samples / self.train_generator.batch_size))
        self.validation_steps = int(np.ceil(self.valid_generator.samples / self.valid_generator.batch_size))

        print(self.steps_per_epoch)
        print(self.validation_steps)

        
        # train_gen = self.train_generator.repeat()
        # valid_gen = self.valid_generator.repeat()

        self.model.fit(
            self.train_dataset,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_dataset,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.training_model_path,
            model=self.model
        )

In [75]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )

except Exception as e:
    raise e

[2024-09-19 16:34:20,291: INFO: common: yaml file: config\config.yaml]
[2024-09-19 16:34:20,293: INFO: common: yaml file: params.yaml]
[2024-09-19 16:34:20,294: INFO: common: Created directory at:artifacts]
[2024-09-19 16:34:20,295: INFO: common: Created directory at:artifacts\prepare_callbacks\checkpoint_dir]
[2024-09-19 16:34:20,295: INFO: common: Created directory at:artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-09-19 16:34:20,296: INFO: common: Created directory at:artifacts\training]
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
39
10
Epoch 1/5
39/39 ━━━━━━━━━━━━━━━━━━━━ 30s 745ms/step - accuracy: 0.5639 - loss: 11.1973 - val_accuracy: 0.7821 - val_loss: 1.4580
Epoch 2/5
39/39 ━━━━━━━━━━━━━━━━━━━━ 31s 790ms/step - accuracy: 0.8746 - loss: 1.4583 - val_accuracy: 0.9103 - val_loss: 0.7252
Epoch 3/5
39/39 ━━━━━━━━━━━━━━━━━━━━ 31s 806ms/step - accuracy: 0.8472 - loss: 2.1789 - val_accuracy: 0.8974 - val_loss: 0.7342
Epoch 4/5
39/39 ━━━━━━━━━